In [1]:
import PySimpleGUI as sg
from pygame import mixer, time
import pygame
from pydub import AudioSegment
from pydub.playback import play
import numpy as np
import io
import os
import sounddevice as sd
import librosa

pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
sg.theme('GreenTan')

'GreenTan'

In [3]:
def win():
    layout=[[sg.Text("Choose a status"),sg.Button('Wav Pitch'),
            sg.Button('MIC Pitch')]]
    return sg.Window('GUI',layout,relative_location=(0,-100),finalize=True)

In [4]:
def win1():
    layout=[[sg.Text("Choose a file: "), sg.Input(), sg.FileBrowse(key="-SOUND_PATH-")],
    [sg.Text(size=(12,1), key='-STATUS-')],
    [sg.Button('Play', pad=(10, 0), key='-PLAY-'),sg.Button('Stop', pad=(10, 0), key='-STOP-'),
    sg.Text('Pitch: '),sg.Slider(range=(-12,12), orientation='h', size=(50, 20), enable_events=True, key = '-PITCH-', default_value= 1.0,resolution=1.0)
    ]]
    return sg.Window('WAV FILE',layout,relative_location=(0,-100),finalize=True)

In [5]:
def win2():
    layout=[[sg.Text("MIC PITCH SHIFTER")],
    sg.Text('Pitch: '),sg.Slider(range=(-12,12), orientation='h', size=(50, 20), enable_events=True, key = '-PITCH-', default_value= 1.0,resolution=1.0)
    ]
    return sg.Window('MIC PITCH',layout,relative_location=(0,-100),finalize=True)

In [6]:
def change_pitch(audio, semitones):
    return audio._spawn(audio.raw_data, overrides={
        "frame_rate": int(audio.frame_rate * (2 ** (semitones / 12.0)))
    })

In [7]:
mixer.init()
pygame.init()
is_playing = False

In [8]:
def pitch():
    window_pitch = win1()
    temp_file_path = "" 
    try:
        while True:
            event, values = window_pitch.read()
            if event == sg.WIN_CLOSED or event == "Exit":
                mixer.quit()
                temp_file_path = values['-SOUND_PATH-'] + 'temp.wav'
                break
            sound_path = values["-SOUND_PATH-"]
            if not sound_path:
                sg.Popup("No song specified.")
                continue
            pitch_value = values['-PITCH-']
            audio = AudioSegment.from_file(sound_path)
            modified_audio = change_pitch(audio, pitch_value)
            if event == '-PLAY-':
                window_pitch['-STATUS-'].update('Playing')
                is_playing = True
                mixer.music.stop()
                pygame.mixer.quit()
                temp_file_path = sound_path + 'temp.wav'
                modified_audio.export(temp_file_path, format="wav")
                mixer.init()
                mixer.music.load(temp_file_path)
                mixer.music.play()
            elif event == '-STOP-':
                mixer.music.stop()
                is_playing = False
    finally:
        window_pitch.close()
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)

In [9]:
def mic():
    window_mic=win2()
    while True:
        event, values = window_mic.read()
        if event == sg.WIN_CLOSED or event=="Exit":
            mixer.quit()
            temp_file_path = sound_path+'temp.wav'
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)
        break
        sound_path = values["-SOUND_PATH-"]
        if not sound_path:
            sg.Popup("No song specificed.")
            continue
        pitch_value = values['-PITCH-']
        audio = AudioSegment.from_file(sound_path)
        modified_audio = change_pitch(audio, pitch_value)
        if event == '-PLAY-':
            window['-STATUS-'].update('Playing')
            is_playing = True
            modified_audio.export(sound_path+'temp.wav', format="wav")
            mixer.music.load(sound_path+'temp.wav')
            mixer.music.play()
        elif event == '-STOP-':
            mixer.music.stop()
            is_playing = False

    window_mic.close()

In [10]:
window_main=win()
while True:
        event, values = window_main.read()
        if event in (None, 'Quit'):
            break
        if event =='Wav Pitch':
            try:
                pitch()
            except:
                sg.popup('Instance Ended, Continuing to wait')
        elif event == 'MIC Pitch':
            try:
                mic()
            except:
                sg.popup('Instance Ended, Continuing to wait')
window.close()

NameError: name 'window' is not defined